In [1]:
from functools import reduce
import pickle
import os

import pandas as pd
import numpy as np

from sklearn.metrics import ndcg_score, average_precision_score
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer

from clearml import Task

from evaluation import generate_recommendations, evaluate_recommendations

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
task = Task.init(
    project_name = 'MoviesGRS_MFDP', 
    task_name = 'PlotEmbeddingsRecommender',
    tags = ['PlotEmbeddings', 'Evaluation', 'TimeSeriesSplit']
)

ClearML Task: created new task id=b65008518cb14b828563ec01d007a72f
2023-05-30 12:27:41,593 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/f3cb8157bfe7443abdc531a44bb15332/experiments/b65008518cb14b828563ec01d007a72f/output/log


In [3]:
data_path = 'data/'
train_data = pd.read_parquet(data_path + 'ratings_train.pq')
test_data = pd.read_parquet(data_path + 'ratings_test.pq')
movies_data = pd.read_parquet(data_path + 'movies_train.pq')
groups2 = pd.read_parquet(data_path + 'groups2.pq')
groups3 = pd.read_parquet(data_path + 'groups3.pq')
groups4 = pd.read_parquet(data_path + 'groups4.pq')
groups5 = pd.read_parquet(data_path + 'groups5.pq')
groups6 = pd.read_parquet(data_path + 'groups6.pq')
groups7 = pd.read_parquet(data_path + 'groups7.pq')

groups_list = [f'group{i}' for i in range(2, 8)]
for i, group in enumerate([groups2, groups3, groups4, groups5, groups6, groups7]):
    test_data = test_data.merge(group, on='userId').rename(columns={'group': f'group{i+2}'})
del groups2, groups3, groups4, groups5, groups6, groups7
test_data

,userId,movieId,rating,group2,group3,group4,group5,group6,group7
0,41988,790,4.0,12988,9525,27113,31361,14281,11298
1,41988,524,3.0,12988,9525,27113,31361,14281,11298
2,41988,608,4.0,12988,9525,27113,31361,14281,11298
3,41988,695,3.0,12988,9525,27113,31361,14281,11298
4,41988,566,4.0,12988,9525,27113,31361,14281,11298
...,...,...,...,...,...,...,...,...,...
3596596,7343,164,5.0,28907,7252,4507,17403,2054,4080
3596597,7343,193,4.0,28907,7252,4507,17403,2054,4080
3596598,7343,253,4.0,28907,7252,4507,17403,2054,4080
3596599,7343,483,2.0,28907,7252,4507,17403,2054,4080


In [4]:
movie_ids = train_data.movieId.unique()

In [5]:
unwatched = (
    train_data
    .groupby(by='userId')
    .agg({'movieId': list})
    .reset_index()
)
unwatched['unwatched'] = (
    unwatched.movieId
    .apply(
        lambda x: movie_ids[
            np.isin(movie_ids, x, invert=True)
        ]
    )
)

unwatched = (
    test_data
    .merge(unwatched[['userId', 'unwatched']], on='userId')
    [['userId', *groups_list, 'unwatched']]
    .groupby(by=['userId', *groups_list])
    .unwatched
    .first()
    .reset_index()
)
unwatched

,userId,group2,group3,group4,group5,group6,group7,unwatched
0,1,14465,39625,6774,4424,23830,7737,"[459, 310, 18, 108, 57, 228, 16, 291, 15, 191,..."
1,2,10418,7556,11348,18784,2598,13437,"[459, 18, 108, 57, 228, 16, 786, 291, 15, 191,..."
2,3,53801,35849,12281,29761,20828,14279,"[459, 310, 519, 18, 108, 57, 228, 786, 291, 15..."
3,4,3990,24019,11784,12577,17824,10523,"[459, 310, 519, 18, 108, 57, 786, 291, 15, 191..."
4,5,39404,9579,25927,22262,607,10989,"[459, 310, 519, 18, 108, 57, 228, 16, 786, 291..."
...,...,...,...,...,...,...,...,...
128583,128660,22670,42046,12206,32028,27049,10083,"[459, 310, 519, 18, 108, 57, 228, 16, 786, 291..."
128584,128661,30264,32218,16183,22934,22798,20087,"[459, 310, 18, 108, 57, 228, 16, 291, 15, 191,..."
128585,128662,24301,22982,420,23328,26324,7171,"[459, 519, 18, 108, 57, 228, 16, 786, 291, 15,..."
128586,128663,55355,18586,11297,13396,19445,7989,"[459, 310, 18, 108, 57, 228, 16, 786, 291, 15,..."


In [6]:
users_watch_history_test: pd.DataFrame = (
    test_data
    .sort_values(by='rating', ascending=False)
    .groupby(by='userId')
    .agg(
        {
        **{g: 'first' for g in groups_list},
        'movieId': list,
        'rating': list
        }
    )
    .reset_index()
)
users_watch_history_test['movieId'] = users_watch_history_test.movieId.apply(np.array)
users_watch_history_test['rating'] = users_watch_history_test.rating.apply(np.array)
users_watch_history_test

,userId,group2,group3,group4,group5,group6,group7,movieId,rating
0,1,14465,39625,6774,4424,23830,7737,"[384, 613, 181, 16, 114, 533, 131, 584, 572, 4...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, ..."
1,2,10418,7556,11348,18784,2598,13437,"[647, 540, 609, 801, 572, 716, 548, 689, 711, ...","[5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.5, 4.5, ..."
2,3,53801,35849,12281,29761,20828,14279,"[51, 230, 62, 80, 100, 115, 144, 50, 55, 141, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, ..."
3,4,3990,24019,11784,12577,17824,10523,"[79, 239, 80, 74, 242, 135, 54, 96, 0, 227, 17...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, ..."
4,5,39404,9579,25927,22262,607,10989,"[225, 67, 133, 217, 191, 45, 75, 175, 98, 142,...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, ..."
...,...,...,...,...,...,...,...,...,...
128583,128660,22670,42046,12206,32028,27049,10083,"[100, 59, 204, 461, 334, 68, 130, 284, 549, 73...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ..."
128584,128661,30264,32218,16183,22934,22798,20087,"[90, 516, 354, 653, 714, 513, 15, 638, 42, 791...","[4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, ..."
128585,128662,24301,22982,420,23328,26324,7171,"[167, 186, 68, 134, 251, 82, 154, 157, 496, 495]","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 4.0, 3.0, ..."
128586,128663,55355,18586,11297,13396,19445,7989,"[524, 695, 664, 656, 715, 488, 674, 561, 630, ...","[5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.0, 4.0, ..."


In [7]:
movies_data = movies_data[['movieId', 'plot']]
movies_data

,movieId,plot
0,0,"A group of living toys, who assume lifelessnes..."
1,1,"In 1986, MI6 agents James Bond and Alec Trevel..."
2,2,Popular Democratic President Andrew Shepherd p...
5,3,"In 1973, sports handicapper and Mafia associat..."
6,4,"When Mr. Dashwood dies, his wife and three dau..."
...,...,...
4152,797,"In 1991, the brainwashed super-soldier James ""..."
4180,798,"In 1877, bounty hunter and Union veteran Major..."
4255,799,"In 2035, the crew of the Ares III mission to M..."
4266,800,Within the mind of a young girl named Riley ar...


In [8]:
BigBadModel = SentenceTransformer('paraphrase-distilroberta-base-v1')

movies_data['embedding'] = movies_data['plot'].apply(BigBadModel.encode)
movies_data

2023-05-30 12:28:03,165 - clearml.model - INFO - Selected model id: edda0fad26674d208e01879b3a5cf727


,movieId,plot,embedding
0,0,"A group of living toys, who assume lifelessnes...","[-0.1895642, 0.39579535, 0.101590574, -0.13271..."
1,1,"In 1986, MI6 agents James Bond and Alec Trevel...","[0.152432, 0.22731969, 0.35638723, 0.57620895,..."
2,2,Popular Democratic President Andrew Shepherd p...,"[-0.4052123, 0.51854575, -0.09895681, -0.24726..."
5,3,"In 1973, sports handicapper and Mafia associat...","[0.006261606, -0.026158923, 0.09536111, 0.2737..."
6,4,"When Mr. Dashwood dies, his wife and three dau...","[-0.30718213, 0.70415014, 0.5588496, 0.6722348..."
...,...,...,...
4152,797,"In 1991, the brainwashed super-soldier James ""...","[-0.0211845, 0.45003378, 0.21097228, -0.355851..."
4180,798,"In 1877, bounty hunter and Union veteran Major...","[-0.02784963, 0.123887956, 0.19217172, 0.37382..."
4255,799,"In 2035, the crew of the Ares III mission to M...","[-0.2835912, -0.13103433, 0.009648345, 0.14046..."
4266,800,Within the mind of a young girl named Riley ar...,"[-0.10373472, 0.5093446, 0.3733033, 0.48482883..."


In [9]:
users_data = train_data.groupby('userId').agg({col: list for col in ['movieId', 'rating']}).reset_index()
users_data['movieId'] = users_data['movieId'].apply(np.array)
users_data['rating'] = users_data['rating'].apply(np.array)
users_data

,userId,movieId,rating
0,0,"[592, 406, 460, 27, 322, 90, 461, 474, 213, 50...","[5.0, 4.5, 3.0, 4.5, 4.5, 5.0, 3.0, 3.5, 5.0, ..."
1,1,"[531, 626, 801, 729, 281, 589, 600, 701, 587, ...","[3.5, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.5, ..."
2,2,"[167, 445, 135, 175, 723, 170, 754, 166, 51, 4...","[4.0, 4.0, 4.0, 3.0, 5.0, 3.5, 0.5, 4.5, 3.5, ..."
3,3,"[92, 145, 114, 68, 26, 122, 20, 214, 148, 149,...","[3.0, 3.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, ..."
4,4,"[243, 197, 59, 234, 115, 251, 82, 55, 230, 256...","[5.0, 4.0, 1.0, 5.0, 2.0, 5.0, 3.0, 5.0, 5.0, ..."
...,...,...,...
128660,128660,"[248, 497, 494, 443, 265, 527, 407, 394, 122, ...","[4.0, 4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 4.0, ..."
128661,128661,"[550, 684, 367, 597, 721, 575, 11, 313, 593, 7...","[4.5, 3.0, 2.5, 4.0, 2.0, 4.5, 4.0, 2.0, 3.5, ..."
128662,128662,"[31, 112, 200, 91, 172, 174, 310, 169, 22, 162...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 4.0, ..."
128663,128663,"[608, 504, 590, 711, 665, 627, 677, 589, 569, ...","[4.0, 4.0, 4.5, 5.0, 3.0, 4.0, 5.0, 4.5, 4.5, ..."


In [10]:
users_data['embedding'] = users_data.apply(
    lambda row:
    np.mean(movies_data[movies_data.movieId.isin(row['movieId'])].embedding * row['rating'], axis=0),
    axis=1
)
users_data = users_data[['userId', 'embedding']]
users_data

,userId,embedding
0,0,"[-0.28446156, 0.93299216, 0.5715234, 0.64569, ..."
1,1,"[-0.30260557, 0.8409, 0.59351224, 0.42042017, ..."
2,2,"[-0.2816124, 0.6678724, 0.59545726, 0.37789539..."
3,3,"[-0.3935022, 0.86840427, 0.4956644, 0.38940465..."
4,4,"[-0.066473536, 0.85598683, 0.58889, 0.50519514..."
...,...,...
128660,128660,"[-0.4618346, 1.0923915, 0.19005956, 0.64650476..."
128661,128661,"[-0.17767367, 1.058262, 0.6296759, 0.50269556,..."
128662,128662,"[-0.34363356, 0.28235617, 0.73458624, 1.049384..."
128663,128663,"[-0.44386974, 0.7305702, 0.53965324, 0.4440984..."


## Average user

In [11]:
def recommend(row):
    unwatched_films = movies_data[
        movies_data.movieId.isin(row['unwatched'])
    ].reset_index(drop=True)
    avg_user = np.mean(
        users_data[
            users_data.userId.isin(row['userId'])
        ].embedding,
        axis=0
    )


    nbrs = NearestNeighbors(n_neighbors=10)
    nbrs.fit(
        np.stack(
            unwatched_films.embedding.values
        )
    )
    _, top_movies_ind = nbrs.kneighbors([avg_user])
    top_movies = unwatched_films.loc[top_movies_ind[0], 'movieId'].values
    return top_movies

In [12]:
recommends = generate_recommendations(recommend, users_watch_history_test, unwatched, groups_list)
recommends

,userId,group2,group3,group4,group5,group6,group7,movieId,rating,group2_rec,group3_rec,group4_rec,group5_rec,group6_rec,group7_rec
0,1,14465,39625,6774,4424,23830,7737,"[384, 613, 181, 16, 114, 533, 131, 584, 572, 4...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, ...","[57, 204, 361, 517, 315, 417, 376, 259, 462, 496]","[315, 95, 325, 57, 248, 130, 337, 417, 597, 408]","[517, 315, 167, 361, 95, 417, 337, 496, 649, 130]","[315, 517, 408, 417, 204, 287, 57, 325, 513, 130]","[204, 417, 57, 517, 652, 167, 74, 130, 248, 496]","[204, 517, 57, 417, 652, 337, 95, 361, 283, 611]"
1,5104,48420,12472,25594,30892,8245,7737,"[208, 85, 8, 290, 198, 206, 172, 184, 112, 195...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[417, 408, 325, 517, 376, 461, 204, 462, 57, 259]","[57, 517, 669, 337, 461, 417, 571, 596, 749, 408]","[517, 204, 571, 417, 596, 163, 376, 461, 259, ...","[517, 204, 461, 571, 669, 74, 417, 408, 596, 652]","[417, 408, 693, 337, 462, 361, 325, 596, 163, ...","[204, 517, 57, 417, 652, 337, 95, 361, 283, 611]"
2,109704,31819,26528,10792,15512,26177,7737,"[176, 59, 458, 164, 462, 114, 689, 353, 14, 43...","[5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.5, 4.5, ...","[517, 669, 361, 749, 461, 693, 57, 315, 417, 462]","[315, 417, 517, 95, 259, 204, 274, 361, 512, 248]","[517, 417, 315, 57, 461, 693, 130, 571, 337, 287]","[204, 693, 461, 417, 337, 57, 517, 361, 706, 669]","[517, 315, 417, 130, 57, 462, 408, 669, 693, 337]","[204, 517, 57, 417, 652, 337, 95, 361, 283, 611]"
3,128307,463,31895,6380,19099,24828,7737,"[97, 244, 33, 142, 80, 99, 181, 223, 176, 157,...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...","[204, 517, 749, 652, 669, 376, 417, 461, 693, ...","[517, 749, 693, 669, 652, 461, 376, 417, 46, 337]","[693, 204, 749, 669, 517, 54, 236, 496, 361, 283]","[517, 417, 693, 652, 749, 669, 376, 283, 337, ...","[57, 517, 749, 417, 461, 693, 669, 337, 95, 408]","[204, 517, 57, 417, 652, 337, 95, 361, 283, 611]"
4,32850,15425,9692,2595,281,7953,7737,"[55, 394, 730, 384, 90, 548, 793]","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 3.5]","[204, 693, 741, 376, 444, 139, 749, 57, 517, 130]","[517, 693, 417, 57, 259, 741, 462, 461, 315, 408]","[693, 517, 669, 74, 57, 749, 315, 408, 559, 283]","[204, 517, 693, 417, 315, 461, 669, 325, 361, ...","[517, 693, 204, 417, 57, 669, 408, 741, 462, 361]","[204, 517, 57, 417, 652, 337, 95, 361, 283, 611]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128583,98075,55317,35155,28186,15593,19307,16497,"[51, 353, 707, 0, 625, 199, 786, 510, 167, 644...","[5.0, 4.0, 4.0, 4.0, 4.0, 3.5, 3.5, 3.0, 3.0, ...","[517, 693, 376, 337, 130, 461, 57, 74, 361, 417]","[517, 204, 693, 376, 461, 57, 74, 259, 337, 596]","[517, 57, 669, 74, 361, 315, 417, 130, 95, 408]","[693, 517, 669, 57, 715, 513, 564, 337, 95, 361]","[669, 376, 361, 417, 749, 130, 95, 283, 652, 741]","[517, 462, 315, 337, 596, 57, 130, 163, 693, 204]"
128584,1764,6389,1952,29114,1967,6059,16497,"[584, 462, 55, 519, 695, 181, 662, 68, 166, 62...","[5.0, 4.5, 4.5, 4.5, 4.0, 4.0, 4.0, 4.0, 3.5, ...","[337, 461, 204, 315, 376, 130, 596, 287, 274, ...","[496, 675, 461, 315, 462, 408, 564, 74, 57, 337]","[461, 337, 517, 259, 693, 596, 287, 315, 462, 46]","[376, 517, 204, 462, 325, 461, 315, 259, 95, 669]","[461, 462, 376, 517, 596, 287, 669, 325, 204, ...","[517, 462, 315, 337, 596, 57, 130, 163, 693, 204]"
128585,18641,5515,7461,18994,10646,4255,20065,"[376, 199, 68, 99, 310, 572, 535, 75, 548, 443...","[5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.0, 4.0, 4.0, ...","[204, 669, 167, 496, 57, 693, 715, 630, 564, 400]","[204, 669, 74, 57, 376, 693, 361, 417, 715, 496]","[669, 57, 496, 517, 462, 417, 408, 361, 400, 248]","[669, 517, 325, 749, 337, 417, 715, 57, 315, 462]","[204, 517, 669, 57, 693, 167, 361, 749, 337, 652]","[204, 715, 315, 496, 417, 531, 620, 517, 408, 57]"
128586,46254,44498,8562,3795,15078,21003,20065,"[540, 561, 777, 513, 59, 707, 766, 715, 334, 7...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.5, ...","[57, 517, 693, 315, 461, 417, 1

In [13]:
avg_user_results = evaluate_recommendations(recommends, groups_list)
avg_user_results

,MAP,NDCG
group2,0.025153,0.199802
group3,0.021447,0.177320
group4,0.019228,0.163409
group5,0.019639,0.165260
group6,0.018149,0.155263
group7,0.016837,0.146662


## Group sum

In [14]:
def recommend(row):
    unwatched_films = movies_data[
        movies_data.movieId.isin(row['unwatched'])
    ].reset_index(drop=True)
    users = np.stack(
        users_data[
            users_data.userId.isin(row['userId'])
        ].embedding.values
    )

    nbrs = NearestNeighbors(n_neighbors=10)
    nbrs.fit(
        np.stack(
            unwatched_films.embedding.values
        )
    )
    probs = {}
    prob, ind = nbrs.kneighbors(users)
    for p, ix in zip(prob.flatten(), ind.flatten()):
        probs[ix] = probs.get(ix, 0) + p
    top_movies_ind = sorted(probs.keys(), key=lambda x: -probs[x])[:10]
    top_movies = unwatched_films.loc[top_movies_ind, 'movieId'].values
    return top_movies

In [15]:
recommends = generate_recommendations(recommend, users_watch_history_test, unwatched, groups_list)
recommends

,userId,group2,group3,group4,group5,group6,group7,movieId,rating,group2_rec,group3_rec,group4_rec,group5_rec,group6_rec,group7_rec
0,1,14465,39625,6774,4424,23830,7737,"[384, 613, 181, 16, 114, 533, 131, 584, 572, 4...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, ...","[315, 517, 361, 204, 57, 236, 649, 376, 597, 496]","[95, 315, 248, 325, 462, 74, 417, 57, 596, 796]","[95, 361, 315, 517, 167, 417, 325, 236, 400, 649]","[417, 517, 315, 325, 287, 408, 204, 274, 252, ...","[204, 315, 130, 517, 496, 57, 510, 611, 385, 274]","[417, 517, 204, 57, 361, 337, 652, 95, 163, 611]"
1,5104,48420,12472,25594,30892,8245,7737,"[208, 85, 8, 290, 198, 206, 172, 184, 112, 195...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[517, 325, 408, 417, 163, 337, 512, 274, 596, ...","[337, 517, 408, 461, 417, 54, 749, 669, 57, 434]","[517, 337, 163, 596, 571, 461, 417, 204, 512, ...","[517, 571, 417, 204, 596, 693, 669, 408, 461, 74]","[337, 417, 715, 361, 693, 325, 408, 596, 259, ...","[417, 517, 204, 57, 361, 337, 652, 95, 163, 611]"
2,109704,31819,26528,10792,15512,26177,7737,"[176, 59, 458, 164, 462, 114, 689, 353, 14, 43...","[5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.5, 4.5, ...","[517, 571, 287, 259, 408, 462, 461, 417, 512, ...","[361, 417, 315, 95, 274, 512, 204, 571, 259, 517]","[417, 517, 571, 130, 57, 408, 315, 287, 337, 461]","[259, 461, 693, 204, 29, 166, 630, 458, 724, 706]","[517, 130, 315, 417, 462, 571, 57, 503, 408, 652]","[417, 517, 204, 57, 361, 337, 652, 95, 163, 611]"
3,128307,463,31895,6380,19099,24828,7737,"[97, 244, 33, 142, 80, 99, 181, 223, 176, 157,...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...","[669, 517, 204, 283, 723, 130, 385, 693, 652, ...","[669, 517, 693, 417, 376, 749, 461, 531, 283, ...","[669, 749, 204, 693, 517, 283, 236, 797, 54, 496]","[693, 517, 652, 417, 669, 376, 283, 749, 531, ...","[461, 749, 669, 693, 517, 57, 417, 283, 337, 596]","[417, 517, 204, 57, 361, 337, 652, 95, 163, 611]"
4,32850,15425,9692,2595,281,7953,7737,"[55, 394, 730, 384, 90, 548, 793]","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 3.5]","[741, 693, 204, 517, 130, 513, 408, 417, 749, ...","[417, 693, 517, 57, 315, 462, 259, 337, 461, 531]","[517, 57, 693, 669, 74, 408, 461, 749, 559, 259]","[693, 204, 517, 741, 417, 57, 315, 669, 325, 461]","[517, 417, 693, 204, 259, 315, 408, 669, 462, ...","[417, 517, 204, 57, 361, 337, 652, 95, 163, 611]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128583,98075,55317,35155,28186,15593,19307,16497,"[51, 353, 707, 0, 625, 199, 786, 510, 167, 644...","[5.0, 4.0, 4.0, 4.0, 4.0, 3.5, 3.5, 3.0, 3.0, ...","[130, 337, 517, 417, 564, 741, 791, 57, 376, 693]","[517, 74, 461, 204, 417, 57, 376, 693, 251, 652]","[57, 517, 130, 74, 417, 669, 315, 95, 361, 741]","[693, 57, 715, 517, 337, 664, 277, 531, 221, 797]","[361, 669, 130, 417, 376, 465, 749, 652, 283, ...","[204, 163, 408, 462, 57, 315, 517, 130, 337, 46]"
128584,1764,6389,1952,29114,1967,6059,16497,"[584, 462, 55, 519, 695, 181, 662, 68, 166, 62...","[5.0, 4.5, 4.5, 4.5, 4.0, 4.0, 4.0, 4.0, 3.5, ...","[461, 337, 513, 29, 544, 325, 757, 517, 204, 376]","[564, 400, 496, 461, 315, 573, 74, 434, 620, 675]","[517, 596, 259, 461, 337, 57, 693, 462, 287, 315]","[204, 517, 376, 462, 596, 269, 167, 283, 741, 46]","[462, 461, 596, 669, 325, 376, 95, 204, 517, 130]","[204, 163, 408, 462, 57, 315, 517, 130, 337, 46]"
128585,18641,5515,7461,18994,10646,4255,20065,"[376, 199, 68, 99, 310, 572, 535, 75, 548, 443...","[5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.0, 4.0, 4.0, ...","[496, 167, 669, 204, 313, 564, 510, 277, 57, 693]","[204, 361, 167, 400, 496, 669, 417, 630, 376, ...","[57, 669, 496, 417, 517, 400, 462, 361, 202, 359]","[669, 417, 715, 462, 517, 337, 749, 57, 315, 259]","[204, 167, 57, 517, 693, 361, 669, 337, 496, 531]","[417, 496, 715, 204, 315, 57, 462, 337, 248, 326]"
128586,46254,44498,8562,3795,15078,21003,20065,"[540, 561, 777, 513, 59, 707, 766, 715, 334, 7...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.5, ...","[417, 461, 315, 517, 57, 749,

In [16]:
group_sum_results = evaluate_recommendations(recommends, groups_list)
group_sum_results

,MAP,NDCG
group2,0.022708,0.185084
group3,0.020333,0.169555
group4,0.018708,0.159155
group5,0.018897,0.160260
group6,0.017627,0.151329
group7,0.016486,0.143736


In [17]:
task.upload_artifact('avg_user_metrics', avg_user_results)
task.upload_artifact('group_sum_metrics', group_sum_results)

True

In [18]:
task.close()